In [12]:
import numpy as np
import pandas as pd
import os
from skimage import io

In [13]:
root_dir = '../../data/komonjo_experiment/200003076/'
image_dir = root_dir + 'training_data/images/'
annotation_data_path = '../../data/200003076/200003076_coordinate.csv'
annotation_data = pd.read_csv(annotation_data_path)
output_dir = root_dir + 'training_data/one_hot_x0.9/'

In [14]:
def get_utf16_hex_and_bounding_box_list_from_csv(image_name, annotation_csv):
    image_name, _ = image_name.split('.')
    annotation_data_for_the_image = annotation_csv[annotation_csv.Image == image_name]
    utf16_hex_and_bounding_box_list = annotation_data_for_the_image[['Unicode', 
                                                                   'X', 'Y', 'Width', 'Height']].values.tolist()
    return utf16_hex_and_bounding_box_list

In [15]:
# if unicode means hiragana return 1 else if kanji return 2
def get_channel_index_from_utf16_hex(char_utf16_hex):
    min_hiragana_int = int('3041', 16) # ぁ
    max_hiragana_int = int('309D', 16) # ゝ
    
    char_int = int(char_utf16_hex[2:], 16)
    
    if min_hiragana_int <= char_int and char_int <= max_hiragana_int:
        return 1
    else:
        return 2

In [16]:
def revise_bounding_box_by_coefficient(X, Y, Width, Height, λ=1.0):
    shrink_rate = 1.0-λ
    shrink_Width = shrink_rate * Width
    shrink_Height = shrink_rate * Height
        
    X = round(X + 0.5*shrink_Width)
    Y = round(Y + 0.5*shrink_Height)
    Width = round(Width - shrink_Width)
    Height = round(Height - shrink_Height)
    return X, Y, Width, Height

In [17]:
def make_one_hot_label_for_a_page(image_dir, image_name, annotation_csv, λ=1.0):
    image_size = io.imread(image_dir + image_name).shape
    one_hot_label_np = np.zeros((image_size[0], image_size[1], 3))
    utf16_hex_and_bounding_box_list = get_utf16_hex_and_bounding_box_list_from_csv(image_name, annotation_csv)
    
    for utf16_hex, X, Y, Width, Height in utf16_hex_and_bounding_box_list:
        Channel = get_channel_index_from_utf16_hex(utf16_hex)
        X, Y, Width, Height = revise_bounding_box_by_coefficient(X, Y, Width, Height, λ)
        one_hot_label_np[Y:Y+Height, X:X+Width, Channel] = 1
    
    return (one_hot_label_np*255).astype(np.uint8)

In [ ]:
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    
image_name_list = os.listdir(image_dir)
for image_name in image_name_list:
    label = make_one_hot_label_for_a_page(image_dir, image_name, annotation_data, 0.9)
    io.imsave(output_dir+image_name, label)

/home/kaito/.local/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: ../../data/komonjo_experiment/200003076/training_data/one_hot_x1.0/200003076_00082_2.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/kaito/.local/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: ../../data/komonjo_experiment/200003076/training_data/one_hot_x1.0/200003076_00057_2.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/kaito/.local/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: ../../data/komonjo_experiment/200003076/training_data/one_hot_x1.0/200003076_00133_2.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/kaito/.local/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: ../../data/komonjo_experiment/200003076/training_data/one_hot_x1.0/200003076_00177_2.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/kaito/.local/lib/python3.5/sit

/home/kaito/.local/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: ../../data/komonjo_experiment/200003076/training_data/one_hot_x1.0/200003076_00007_2.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/kaito/.local/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: ../../data/komonjo_experiment/200003076/training_data/one_hot_x1.0/200003076_00171_2.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/kaito/.local/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: ../../data/komonjo_experiment/200003076/training_data/one_hot_x1.0/200003076_00124_2.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/kaito/.local/lib/python3.5/site-packages/skimage/io/_io.py:140: UserWarning: ../../data/komonjo_experiment/200003076/training_data/one_hot_x1.0/200003076_00022_2.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/kaito/.local/lib/python3.5/sit